In [ ]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
from DOSPT import *
from scipy.constants import Boltzmann #light speed in [m/s], #Planck's constant  [J*s]

#-----Simulation type and parameters------
variable = 'P'
l0lp = 0.5
nb = 13     

#Adim param
t = 1.0
p = 2.6

#CH2 parameters (From McCoy1995)
Sigma = 3.56 * 1.e-10                             #Sigma in [m]
kb = Boltzmann                                      #Boltzmann's constant [J/K]
Epsilon = 70.47 * kb                              #Epsilon in [J]
mch2 = 14.026 *(1. / 1000.)/6.02214076e23        #CH2 mass in [kg]

#Simulation type and time
ens = 'NVT'
Dstep = 16               #frames dump
tstep = 0.001*np.sqrt(mch2 / Epsilon)*Sigma  #time step in second
skip = 1                #If don't want load all frames, can jump frames
fcv = (Sigma / (np.sqrt(mch2 / Epsilon)*Sigma))    #conversion factor lj units to [m/s]
fcx = Sigma                                      #conversion factor lj to SI in [m]
SN = 2.                                        #Symmetry Number: Molecula lineal homonuclear = 2 (Ver:TEMA 1. Termodinámica Estadística)
T   =  t * (Epsilon/kb)
m = np.tile(mch2, nb)  #Atoms mass in kg


#Select working folder
if variable == 'T':
    folder = variable+str(t)
elif variable == 'P':
    folder = variable+str(p)
root =  (f'l0lp{l0lp}/nb{nb}/{folder}/')
os.makedirs(root+'PostProc/', exist_ok=True)

j = 1
for i in np.arange(j,j+20):
    #INPUT
    file_in =  root+(f'lc2PT_{i}.DS{Dstep}.t{t}p{p}.trj')
    print('Procesando:',file_in)
    coords = Trajectory(file_in, 0,skip,fcv,fcx)
    rposi = coords.compute_rposi(m,nb)
    del coords
    vels = Trajectory(file_in, 2,skip,fcv,fcx)

    for dof in ['trn', 'rot', 'vib']:
        if dof == 'trn':
            DOSt, freq = vels.Return_DOS_trn(tstep,Dstep,T,m,nb)

        elif dof == 'rot':
            DOSt, freq, I = vels.Return_DOS_rot(rposi,tstep,Dstep,T,m,nb)

        elif dof == 'vib':
            DOSt, freq = vels.Return_DOS_vib(rposi,tstep,Dstep,T,m,nb)

        #OUTPUT
        file_out =  root + (f'PostProc/Dos{dof}{ens}_{i}.DS{Dstep}t*{t}p*{p}.txt')
        header = 'freq [1/s], DOSt[s], DOSa[s], DOSb[s], DOSc[s] / I = {}'.format(I) if dof == 'rot' else 'freq [1/s], DOSt[s], DOSx[s], DOSy[s], DOSz[s]'
        out_file = open(file_out, "w")
        np.savetxt(out_file, (np.vstack((freq[:], DOSt[:,0], DOSt[:,1], DOSt[:,2], DOSt[:,3])).T),\
                    fmt='%E',header=header)
        out_file.close()
    print('Procesado:', file_in)
    del vels

Procesando: l0lp0.5/nb13/P2.6/lc2PT_1.DS16.t1.0p2.6.trj
Are going to be loaded: particles coordinates
Trajectory frames=  501
Frames to be loaded=  501
Are going to be loaded: particles velocities
Trajectory frames=  501
Frames to be loaded=  501
